In [1]:
import sys, json, random, os
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.index_builder import IndexBuilder
from datamart.query_manager import QueryManager

In [2]:
es_index = "datamart_noaa"

qm = QueryManager()

Query any indicator has `average` in its description

In [3]:
query_1 = json.dumps({
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "variables.named_entity": "Los Angeles"
                    }
                },
                {
                    "match": {
                        "variables.named_entity": "new york"
                    }
                },
                {
                    "match": {
                        "description": "average"
                    }
                }
            ]
        }
    }
})

In [4]:
hitted_metadatas = qm.search(index=es_index, body=query_1)

print(len(hitted_metadatas))

6


In [5]:
for metadata in hitted_metadatas:
    print("====== HIT a metadata ======")
    print("\n")
    print("====== GET the dataset ======")
    df = qm.get_dataset(metadata=metadata, variables=None, constrains={
        "locations": ["los angeles", "new york"],
        "date_range": {
            "start": "2018-09-23T00:00:00",
            "end": "2018-10-01T00:00:00"
        }
    })
    if df.shape[0] >= 10:
        print(df.iloc[random.sample(range(1, df.shape[0]), 10), :])
    else:
        print(df)
    print("\n\n")

====== HIT a metadata ======


====== GET the dataset ======


Empty DataFrame
Columns: [date, stationid, city, ACSH]
Index: []



====== HIT a metadata ======


====== GET the dataset ======


Empty DataFrame
Columns: [date, stationid, city, ACMC]
Index: []



====== HIT a metadata ======


====== GET the dataset ======


                    date          stationid         city AWND
55   2018-09-29T00:00:00  GHCND:USW00003167  los angeles   25
112  2018-09-26T00:00:00  GHCND:USW00094789     new york   60
120  2018-09-27T00:00:00  GHCND:USW00094789     new york   57
53   2018-09-28T00:00:00  GHCND:USW00093197  los angeles   16
45   2018-09-28T00:00:00  GHCND:USW00003166  los angeles   17
4    2018-09-23T00:00:00  GHCND:USW00023152  los angeles   27
41   2018-09-27T00:00:00  GHCND:USW00023174  los angeles   35
65   2018-09-30T00:00:00  GHCND:USW00023129  los angeles   21
12   2018-09-24T00:00:00  GHCND:USW00023130  los angeles   18
79   2018-10-01T00:00:00  GHCND:USW00093184  los angeles   17



====== HIT a metadata ======


====== GET the dataset ======


Empty DataFrame
Columns: [date, stationid, city, ACMH]
Index: []



====== HIT a metadata ======


====== GET the dataset ======


Empty DataFrame
Columns: [date, stationid, city, ACSC]
Index: []



====== HIT a metadata ======


====== GET the dataset ======


                   date          stationid         city TAVG
99  2018-10-01T00:00:00  GHCND:USR0000CMAL  los angeles  254
28  2018-09-25T00:00:00  GHCND:USR0000CLTU  los angeles  178
42  2018-09-26T00:00:00  GHCND:USR0000CSFD  los angeles  217
14  2018-09-24T00:00:00  GHCND:USR0000CCHI  los angeles  208
77  2018-09-29T00:00:00  GHCND:USW00023129  los angeles  207
2   2018-09-23T00:00:00  GHCND:USR0000CCHB  los angeles  228
52  2018-09-27T00:00:00  GHCND:USR0000CMIL  los angeles  238
69  2018-09-29T00:00:00  GHCND:USR0000CCHI  los angeles  175
51  2018-09-27T00:00:00  GHCND:USR0000CMAL  los angeles  251
83  2018-09-30T00:00:00  GHCND:USR0000CCLE  los angeles  214



